<a href="https://colab.research.google.com/github/MariaPetrovskaya/Data-Science-demo/blob/main/mashine%20learning%20in%20business%20case/mashine_learning_in_business_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np


**Результат** загрузка всех библиотек 

In [ ]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
data0 = df0.copy()
display (data0.head(5))
print (data0.info())
print ('data0', data0.duplicated().sum())
df1 = pd.read_csv('/datasets/geo_data_1.csv')
data1 = df1.copy()
display (data1.head(5))
print (data1.info())
print ('data1', data1.duplicated().sum())
df2 = pd.read_csv('/datasets/geo_data_2.csv')
data2 = df2.copy()
display (data2.head(5))
print (data2.info())
print ('data2', data2.duplicated().sum())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
data0 0


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
data1 0


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
data2 0


**Результат** выгрузка датасетов и проверка на пропуски и явные дубликаты: пропусков и явных дубликатов не выявлено

## Обучение и проверка модели

**Модель для первого региона (датасет geo_data_0)**

In [ ]:
features0 = data0.drop(['product', 'id'], axis=1)
target0 = data0['product'] 
features0_train, features0_valid, target0_train, target0_valid = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)
print('features0_train shape:', features0_train.shape)
print('features0_valid shape:', features0_valid.shape)
print('target0_train shape:', target0_train.shape)
print('target0_valid shape:', target0_valid.shape)


features0_train shape: (75000, 3)
features0_valid shape: (25000, 3)
target0_train shape: (75000,)
target0_valid shape: (25000,)


**Результат** датасет разбит на тренировочную и валидизационную выборки

In [ ]:
model = LinearRegression(normalize=True)
model.fit(features0_train,target0_train) 
predictions0_valid= model.predict(features0_valid) 
result = mean_squared_error(target0_valid, predictions0_valid)**0.5
print("RMSE модели линейной регрессии на валидационной выборке первого датасета:", result)
predictions0_valid = pd.Series(predictions0_valid)
target0_valid = pd.Series(target0_valid)
#print('предсказания на валидационной выборке', predictions0_valid.head())
#print (predictions0_valid.shape)
#print('правильные ответы на валидационной выборке', target0_valid.head())
#print (target0_valid.shape)
print('Проверям количество скважин с одинаковым ID: ', data0['id'].value_counts())
target0_valid_df = target0_valid.to_frame(name='target0_valid')
target0_valid_df = target0_valid_df.reset_index(drop=True)
predictions0_valid_df = predictions0_valid.to_frame(name='predictions0_valid')
result0 = pd.concat([target0_valid_df, predictions0_valid_df], axis= 1 )
print ('Предсказания (predictions ): ', predictions0_valid_df.head())
print(predictions0_valid_df.shape)
print ('Правильные ответы (target): ', target0_valid_df.head())
print(target0_valid_df.shape)
print('Предсказания и правильные ответы в одном датасете: ', result0.head(10))
print(result0.shape)

RMSE модели линейной регрессии на валидационной выборке первого датасета: 37.5794217150813
Проверям количество скважин с одинаковым ID:  fiKDv    2
74z30    2
bsk9y    2
Tdehs    2
bxg6G    2
        ..
sdMKv    1
eRSc1    1
5xhRD    1
lyZfY    1
BjxgP    1
Name: id, Length: 99990, dtype: int64
Предсказания (predictions ):     predictions0_valid
0           95.894952
1           77.572583
2           77.892640
3           90.175134
4           70.510088
(25000, 1)
Правильные ответы (target):     target0_valid
0      10.038645
1     114.551489
2     132.603635
3     169.072125
4     122.325180
(25000, 1)
Предсказания и правильные ответы в одном датасете:     target0_valid  predictions0_valid
0      10.038645           95.894952
1     114.551489           77.572583
2     132.603635           77.892640
3     169.072125           90.175134
4     122.325180           70.510088
5      37.735331           69.127076
6     139.227191          125.106759
7      72.491987           87.643849
8   

**Результат** 
1. RMSE : метрика, которая сообщает нам квадратный корень из средней квадратичной разницы между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже RMSE, тем лучше модель соответствует набору данных. Получается возможное отклонение на 37+тыс. баррелей. Стоит подумать.
2. Проверила количество скважин с одинаковым ID в датасете - несущественное, можно игнорировать. 
3. Предсказания (predictions )и правильные ответы (target) на валидационной выборке собраны в один датасет result0

**Модель для второго региона (датасет geo_data_1)**

In [ ]:
features1 = data1.drop(['product', 'id'], axis=1)
target1 = data1['product'] 
features1_train, features1_valid, target1_train, target1_valid = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)
print('features1_train shape:', features0_train.shape)
print('features1_valid shape:', features0_valid.shape)
print('target1_train shape:', target0_train.shape)
print('target1_valid shape:', target0_valid.shape)
print(data1['id'].value_counts())


features1_train shape: (75000, 3)
features1_valid shape: (25000, 3)
target1_train shape: (75000,)
target1_valid shape: (25000,)
wt4Uk    2
LHZR0    2
5ltQ6    2
bfPNe    2
J1dt7    1
        ..
Pm2Ct    1
LbdPC    1
CPhgE    1
LtsA5    1
uk68n    1
Name: id, Length: 99996, dtype: int64


**Результат** датасет разбит на тренировочную и валидизационную выборки 

In [ ]:
model = LinearRegression(normalize=True)
model.fit(features1_train,target1_train) 
predictions1_valid= model.predict(features1_valid) 
result1 = mean_squared_error(target1_valid, predictions1_valid)**0.5 
print("RMSE модели линейной регрессии на валидационной выборке второго датасета:", result1)
predictions1_valid = pd.Series(predictions1_valid)
target1_valid = pd.Series(target1_valid)
print('Проверям количество скважин с одинаковым ID: ', data1['id'].value_counts())
target1_valid_df = target1_valid.to_frame(name='target1_valid')
target1_valid_df = target1_valid_df.reset_index(drop=True)
predictions1_valid_df = predictions1_valid.to_frame(name='predictions1_valid')
result1 = pd.concat([target1_valid_df, predictions1_valid_df], axis= 1 )
print ('Предсказания (predictions ): ', predictions1_valid_df.head())
print(predictions1_valid_df.shape)
print ('Правильные ответы (target): ', target1_valid_df.head())
print(target1_valid_df.shape)
print('Предсказания и правильные ответы в одном датасете: ', result1.head(10))
print(result1.shape)

RMSE модели линейной регрессии на валидационной выборке второго датасета: 0.8930992867756168
Проверям количество скважин с одинаковым ID:  wt4Uk    2
LHZR0    2
5ltQ6    2
bfPNe    2
J1dt7    1
        ..
Pm2Ct    1
LbdPC    1
CPhgE    1
LtsA5    1
uk68n    1
Name: id, Length: 99996, dtype: int64
Предсказания (predictions ):     predictions1_valid
0           82.663314
1           54.431786
2           29.748760
3           53.552133
4            1.243856
(25000, 1)
Правильные ответы (target):     target1_valid
0      80.859783
1      53.906522
2      30.132364
3      53.906522
4       0.000000
(25000, 1)
Предсказания и правильные ответы в одном датасете:     target1_valid  predictions1_valid
0      80.859783           82.663314
1      53.906522           54.431786
2      30.132364           29.748760
3      53.906522           53.552133
4       0.000000            1.243856
5     110.992147          111.438490
6     137.945408          137.134374
7      84.038886           82.888902
8 

**Результат** 
1. RMSE 0.89 (тыс. баррелей), относительно небольшой. На выводе фактические данные и предсказанные похожи, результаты на этом датасете гораздо более предсказуемы.
2. Проверила количество скважин с одинаковым ID в датасете - несущественное, можно игнорировать. 
3. Предсказания (predictions )и правильные ответы (target) на валидационной выборке собраны в один датасет result1

**Модель для третьего региона (датасет geo_data_2)**

In [ ]:
features2 = data2.drop(['product', 'id'], axis=1)
target2 = data2['product'] 
features2_train, features2_valid, target2_train, target2_valid = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)
print('features2_train shape:', features0_train.shape)
print('features2_valid shape:', features0_valid.shape)
print('target2_train shape:', target0_train.shape)
print('target2_valid shape:', target0_valid.shape)
print(data2['id'].value_counts())


features2_train shape: (75000, 3)
features2_valid shape: (25000, 3)
target2_train shape: (75000,)
target2_valid shape: (25000,)
Vcm5J    2
VF7Jo    2
xCHr8    2
KUPhW    2
OOsLW    1
        ..
Wj6tX    1
fwr2o    1
jAppI    1
2nvJQ    1
ppzre    1
Name: id, Length: 99996, dtype: int64


**Результат** Датасет разбит на тренировочную и валидизационную выборки

In [ ]:
model = LinearRegression(normalize=True)
model.fit(features2_train,target2_train) 
predictions2_valid= model.predict(features2_valid) 
result2 = mean_squared_error(target2_valid, predictions2_valid)**0.5
print("RMSE модели линейной регрессии на валидационной выборке третьего датасета:", result2)
predictions2_valid = pd.Series(predictions2_valid)
target2_valid = pd.Series(target2_valid)
print('Проверям количество скважин с одинаковым ID: ', data2['id'].value_counts())
target2_valid_df = target2_valid.to_frame(name='target2_valid')
target2_valid_df = target2_valid_df.reset_index(drop=True)
predictions2_valid_df = predictions2_valid.to_frame(name='predictions2_valid')
result2 = pd.concat([target2_valid_df, predictions2_valid_df], axis= 1 )
print ('Предсказания (predictions ): ', predictions2_valid_df.head())
print(predictions2_valid_df.shape)
print ('Правильные ответы (target): ', target2_valid_df.head())
print(target2_valid_df.shape)
print('Предсказания и правильные ответы в одном датасете: ', result2.head(10))
print(result2.shape)

RMSE модели линейной регрессии на валидационной выборке третьего датасета: 40.02970873393434
Проверям количество скважин с одинаковым ID:  Vcm5J    2
VF7Jo    2
xCHr8    2
KUPhW    2
OOsLW    1
        ..
Wj6tX    1
fwr2o    1
jAppI    1
2nvJQ    1
ppzre    1
Name: id, Length: 99996, dtype: int64
Предсказания (predictions ):     predictions2_valid
0           93.599633
1           75.105159
2           90.066809
3          105.162375
4          115.303310
(25000, 1)
Правильные ответы (target):     target2_valid
0      61.212375
1      41.850118
2      57.776581
3     100.053761
4     109.897122
(25000, 1)
Предсказания и правильные ответы в одном датасете:     target2_valid  predictions2_valid
0      61.212375           93.599633
1      41.850118           75.105159
2      57.776581           90.066809
3     100.053761          105.162375
4     109.897122          115.303310
5     105.904629          121.939197
6      51.646180          119.053040
7     173.162121           75.396575
8 

**Результат** 
1. RMSE на этом датасете слишком высокий: возможное отклонение на 40+тыс. баррелей. Стоит подумать.
2. Проверила количество скважин с одинаковым ID в датасете - несущественное, можно игнорировать.
3. Предсказания (predictions )и правильные ответы (target) на валидационной выборке собраны в один датасет result1

**Выводы по моделям для разных регионов** 

С точки зрения адекватности модели, только по второму региону есть хорошие метрики, первый и третий - вряд ли модель отражает закономерности


## Подготовка к расчёту прибыли

In [ ]:
probs = 500
selective = 200
invest = 10000000000
price  = 450000 #за 1000 баррелей



**Результат** Ключевые значения для расчётов сохранены в отдельных переменных

In [ ]:
print ('средняя стоимость разработки 1 скважины, при условии что их выбрано 200: ')
print(invest/selective)
print ('достаточный объём сырья для безубыточной разработки новой скважины, ROI = 0, c одной шахты, в тыс. баррелей: ')
print(invest/selective/price)
print ('достаточный объём сырья для безубыточной разработки новой скважины, ROI = 0, суммарный по региону: ')
print (invest/price)

средняя стоимость разработки 1 скважины, при условии что их выбрано 200: 
50000000.0
достаточный объём сырья для безубыточной разработки новой скважины, ROI = 0, c одной шахты, в тыс. баррелей: 
111.11111111111111
достаточный объём сырья для безубыточной разработки новой скважины, ROI = 0, суммарный по региону: 
22222.222222222223


**Результат** Рассчитан достаточный объём сырья для безубыточной разработки новой скважины

In [ ]:
print('средний объем добычи с шахты в регионе 1:  ', data0['product'].mean())
print('суммарный объем по данным региона 1: ', data0['product'].sum())
print()
print('средний объем добычи с шахты в регионе 2:  ', data1['product'].mean())
print('суммарный объем по данным региона 2: ',data1['product'].sum())
print()
print('средний объем добычи с шахты в регионе 3:  ', data2['product'].mean())
print('суммарный объем по данным региона 3: ',data2['product'].sum())

средний объем добычи с шахты в регионе 1:   92.50000000000001
суммарный объем по данным региона 1:  9250000.000000002

средний объем добычи с шахты в регионе 2:   68.82500000000002
суммарный объем по данным региона 2:  6882500.000000002

средний объем добычи с шахты в регионе 3:   95.00000000000004
суммарный объем по данным региона 3:  9500000.000000004


**Результаты** Проведено сравнение со средним запасом в каждом регионе. Предварительные расчеты показывают, что самые большие средние запасы в третьем регионе и далее первый. Во втором регионе средние запасы существенно ниже, но более надежные метрики у модели. 

## Расчёт прибыли и рисков 

**Предварительная оценка** присутствуют ли в достаточном количестве шахты в большими объемами в датасете каждого региона:

In [ ]:
sorted_data0 = data0.sort_values(by='product', ascending=False).head(200)
print(sorted_data0)
print(sorted_data0.shape)

          id        f0        f1        f2     product
8826   rjMou  1.797736  0.098212  6.148260  185.364347
99818  7cHIv  0.518445 -0.412790  4.951916  185.362690
94175  uCDzR  0.351428 -0.400244  7.227618  185.355615
1925   IfqrC  0.624430 -0.469312  5.753677  185.354980
45291  5FEPb  1.758787 -0.395038  6.719085  185.352015
...      ...       ...       ...       ...         ...
34856  Xqm8i  2.038708  0.271003  3.791803  184.366474
54135  oAHG7  1.108500 -0.583939  1.503254  184.362338
63598  CaN8V  1.330398 -0.523232  6.365799  184.360762
4941   orIzp  1.674001  0.016279  3.869249  184.358756
79494  obTWo  1.313813 -0.459792  9.423778  184.356455

[200 rows x 5 columns]
(200, 5)


In [ ]:
sorted_data1 = data1.sort_values(by='product', ascending=False).head(200)
print(sorted_data1)
print(sorted_data1.shape)

          id         f0         f1        f2     product
53864  MzRzn   2.901352  -3.475398  5.001393  137.945408
97083  nW6eC  -4.030430 -14.020643  5.009571  137.945408
88340  xwJzQ  -3.373117  -9.227661  4.994369  137.945408
64879  Xd8DC  -5.360281  -2.388204  5.001439  137.945408
7288   Stdrb  -8.058661  -2.244840  5.000753  137.945408
...      ...        ...        ...       ...         ...
88496  cfKRc  -3.160538  -6.225915  4.999458  137.945408
92616  22reW -11.067158  -0.340721  4.999463  137.945408
41100  oQt7x  -7.557897  -1.563086  5.001729  137.945408
7584   wvAAH  -3.325670  -6.932481  4.998615  137.945408
16041  22GNv -14.171773   2.840246  4.996514  137.945408

[200 rows x 5 columns]
(200, 5)


In [ ]:
sorted_data2 = data2.sort_values(by='product', ascending=False).head(200)
print(sorted_data2)
print(sorted_data2.shape)

          id        f0        f1         f2     product
79705  UAhji -2.747914  1.555227   3.342182  190.029838
93444  IB0JE  3.026506  1.344623   8.891243  190.013589
35099  2HeCn  2.848844 -1.701996   9.437101  190.011722
21943  dldNH -2.500091  2.024392   6.291513  190.010982
37870  CpuBi -0.686667 -3.219452   5.963290  190.010029
...      ...       ...       ...        ...         ...
96482  hfbIz  1.071479 -1.978181   4.437436  189.038807
53814  6YAZG  2.122462 -1.071496   4.818192  189.038746
5138   RO6MB  1.509688  3.125449  11.400160  189.037389
91855  y3w0m  0.415052 -2.409421   2.205990  189.024650
51559  inKEw -2.564780 -1.793263   4.337253  189.024253

[200 rows x 5 columns]
(200, 5)


**Выводы** во всех датесетах присутствует достаточное количество шахт с нужными объемами

**Функции для расчёта прибыли по выбранным скважинам и предсказаниям модели**

In [ ]:
def revenue(target, predictions, count):
    probs_sorted = predictions.sort_values (ascending=False)
    selected = target[probs_sorted.index][:count] 
    return price * selected.sum()# < напишите код здесь >

target = target0_valid_df['target0_valid']
predictions = predictions0_valid_df['predictions0_valid']

res0 = revenue(target, predictions, selective)

print('выручка первого по первому региону', res0)
print('прибыль: (инвестиции) - (выручка по первому региону): ', res0-invest)

выручка первого по первому региону 13320826043.13985
прибыль: (инвестиции) - (выручка по первому региону):  3320826043.1398506


**Результат** Составлена функция расчета прибыли revenue и применена к первому региону.  

In [ ]:
state = np.random.RandomState(12345)
#def bootstap(target, predictions, count):
values = []
count = 0
for i in range(1000):
    target_subsample = target.sample(probs, replace=True, random_state=state) 
    probs_subsample = predictions[target_subsample.index] 
    values.append(revenue(target_subsample, probs_subsample, selective))
    if (revenue(target_subsample, probs_subsample, selective)) < invest: 
        count = count+1 # < напишите код здесь>
   
values = pd.Series(values)
lower0 = (values.quantile(0.025))
upper0 = (values.quantile(0.975))# < напишите код здесь>

mean0 = values.mean()
print("Средняя выручка:", mean0)
print("2.5%-квантиль:", lower0)
print("97.5%-квантиль:", upper0)
print('количество случаев, когда прибыль меньше инвестиций:', count)
print('процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков:', count/1000*100, '%')

print('средняя выручка по первому региону - инвестиции', mean0-invest)
print('нижняя граница 95% интервала прибыль  - инвестиции: ', lower0-invest)
print ('верхняя граница 95% интервала прибыль  - инвестиции: ', upper0-invest)
print ('доверительный интервал 95%:  от',lower0-invest, '  до  ', upper0-invest )

Средняя выручка: 10425938526.910593
2.5%-квантиль: 9897909905.162064
97.5%-квантиль: 10947976353.35837
количество случаев, когда прибыль меньше инвестиций: 60
процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков: 6.0 %
средняя выручка по первому региону - инвестиции 425938526.91059303
нижняя граница 95% интервала прибыль  - инвестиции:  -102090094.8379364
верхняя граница 95% интервала прибыль  - инвестиции:  947976353.3583698
доверительный интервал 95%:  от -102090094.8379364   до   947976353.3583698


**Результат** На первом датасете применена функция Bootstrap с 1000 выборок, расчитана средняя прибыль, 95%-й доверительный интервал и риск убытков. 

на уровне квартиля 2.5% сумма ниже инвестиций 

In [ ]:
target = target1_valid_df['target1_valid']
predictions = predictions1_valid_df['predictions1_valid']

res1 = revenue(target, predictions, selective)

print('выручка по второму региону', res1)
print('прибыль: (инвестиции) - (выручка по второму региону): ', res1-invest)

выручка по второму региону 12415086696.68151
прибыль: (инвестиции) - (выручка по второму региону):  2415086696.681511


**Результат** функция расчета прибыли revenue и применена ко второму региону

In [ ]:
state = np.random.RandomState(12345)
#def bootstap(target, predictions, count):
values1 = []
count1 = 0
for i in range(1000):
    target_subsample = target.sample(probs, replace=True, random_state=state) 
    probs_subsample = predictions[target_subsample.index] 
    values1.append(revenue(target_subsample, probs_subsample, selective))
    if (revenue(target_subsample, probs_subsample, selective)) < invest: 
        count1 = count1+1# < напишите код здесь>
   
values1 = pd.Series(values1)
lower1 = (values1.quantile(0.025))
upper1 = (values1.quantile(0.975))# < напишите код здесь>
mean1 = values1.mean()
print("Средняя выручка:", mean1)
print("2.5%-квантиль:", lower1)
print("97.5%-квантиль:", upper1)
print('количество случаев, когда прибыль меньше инвестиций:', count1)
print('процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков:', count1/1000*100, '%')
print('средняя выручка по второму региону - инвестиции', mean1-invest)
print('нижняя граница 95% интервала прибыль  - инвестиции: ', lower1-invest)
print ('верхняя граница 95% интервала прибыль  - инвестиции: ', upper1-invest)
print ('доверительный интервал 95%:  от',lower1-invest, '  до  ', upper1-invest )

Средняя выручка: 10515222773.44329
2.5%-квантиль: 10068873225.370502
97.5%-квантиль: 10931547591.25705
количество случаев, когда прибыль меньше инвестиций: 10
процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков: 1.0 %
средняя выручка по второму региону - инвестиции 515222773.4432907
нижняя граница 95% интервала прибыль  - инвестиции:  68873225.37050247
верхняя граница 95% интервала прибыль  - инвестиции:  931547591.2570496
доверительный интервал 95%:  от 68873225.37050247   до   931547591.2570496


**Результат** На втором датасете применена функция Bootstrap с 1000 выборок, расчитана средняя прибыль, 95%-й доверительный интервал и риск убытков.

In [ ]:
target = target2_valid_df['target2_valid']
predictions = predictions2_valid_df['predictions2_valid']

res2 = revenue(target, predictions, selective)

print('выручка по третьему региону', res2)
print('прибыль: (инвестиции) - (выручка по третьему региону): ', res2-invest)

выручка по третьему региону 12710349963.599833
прибыль: (инвестиции) - (выручка по третьему региону):  2710349963.5998325


**Результат** функция расчета прибыли revenue и применена к третьему региону

In [ ]:
state = np.random.RandomState(12345)
#def bootstap(target, predictions, count):
values2 = []
count2 = 0
for i in range(1000):
    target_subsample = target.sample(probs, replace=True, random_state=state) 
    probs_subsample = predictions[target_subsample.index] 
    values2.append(revenue(target_subsample, probs_subsample, selective))
    if (revenue(target_subsample, probs_subsample, selective)) < invest: 
        count2 = count2+1 
   
values2 = pd.Series(values2)
lower2 = (values2.quantile(0.025))
upper2 = (values2.quantile(0.975))

mean2 = values2.mean()
print("Средняя выручка:", mean2)
print("2.5%-квантиль:", lower2)
print("97.5%-квантиль:", upper2)
print('количество случаев, когда прибыль меньше инвестиций:', count2)
print('процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков:', count2/1000*100, '%')
print('средняя выручка по третьему региону - инвестиции', mean2-invest)
print('нижняя граница 95% интервала прибыль  - инвестиции: ', lower2-invest)
print ('верхняя граница 95% интервала прибыль  - инвестиции: ', upper2-invest)
print ('доверительный интервал 95%:  от',lower2-invest, '  до  ', upper2-invest )

Средняя выручка: 10435008362.782757
2.5%-квантиль: 9871119452.670212
97.5%-квантиль: 10969706954.180265
количество случаев, когда прибыль меньше инвестиций: 64
процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков: 6.4 %
средняя выручка по третьему региону - инвестиции 435008362.7827568
нижняя граница 95% интервала прибыль  - инвестиции:  -128880547.32978821
верхняя граница 95% интервала прибыль  - инвестиции:  969706954.1802654
доверительный интервал 95%:  от -128880547.32978821   до   969706954.1802654


**Результат** На третьем датасете применена функция Bootstrap с 1000 выборок, расчитана средняя прибыль, 95%-й доверительный интервал и риск убытков.

на уровне квартиля 2.5% сумма ниже инвестиций

**Сравнение регионов**

In [ ]:
print ('первый регион')
print ('функция расчёта прибыли')
print('выручка первого по первому региону', res0)
print('прибыль: (инвестиции) - (выручка по первому региону): ', res0-invest)
print('процедура Bootstrap')
print('средняя выручка по первому региону - инвестиции', mean0-invest)
print('нижняя граница 95% интервала прибыль  - инвестиции: ', lower0-invest)
print ('верхняя граница 95% интервала прибыль  - инвестиции: ', upper0-invest)
print ('доверительный интервал 95%:  от',lower0-invest, '  до  ', upper0-invest )
print('количество случаев, когда прибыль меньше инвестиций:', count)
print('процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков:', count/1000*100, '%')
print()
print('второй регион')
print ('функция расчёта прибыли')
print('выручка по второму региону', res1)
print('прибыль: (инвестиции) - (выручка по второму региону): ', res1-invest)
print('процедура Bootstrap')
print('средняя выручка по второму региону - инвестиции', mean1-invest)
print('нижняя граница 95% интервала прибыль  - инвестиции: ', lower1-invest)
print ('верхняя граница 95% интервала прибыль  - инвестиции: ', upper1-invest)
print ('доверительный интервал 95%:  от',lower1-invest, '  до  ', upper1-invest )
print('количество случаев, когда прибыль меньше инвестиций:', count1)
print('процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков:', count1/1000*100, '%')
print()
print('третий регион')
print ('функция расчёта прибыли')
print('выручка по третьему региону', res2)
print('прибыль: (инвестиции) - (выручка по третьему региону): ', res2-invest)
print('процедура Bootstrap')
print('средняя выручка по третьему региону - инвестиции', mean2-invest)
print('нижняя граница 95% интервала прибыль  - инвестиции: ', lower2-invest)
print ('верхняя граница 95% интервала прибыль  - инвестиции: ', upper2-invest)
print ('доверительный интервал 95%:  от',lower2-invest, '  до  ', upper2-invest )
print('количество случаев, когда прибыль меньше инвестиций:', count2)
print('процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков:', count2/1000*100, '%')

первый регион
функция расчёта прибыли
выручка первого по первому региону 13320826043.13985
прибыль: (инвестиции) - (выручка по первому региону):  3320826043.1398506
процедура Bootstrap
средняя выручка по первому региону - инвестиции 425938526.91059303
нижняя граница 95% интервала прибыль  - инвестиции:  -102090094.8379364
верхняя граница 95% интервала прибыль  - инвестиции:  947976353.3583698
доверительный интервал 95%:  от -102090094.8379364   до   947976353.3583698
количество случаев, когда прибыль меньше инвестиций: 60
процент случаев, когда прибыль меньше инвестиций (на 1000 итераций) - риск убытков: 6.0 %

второй регион
функция расчёта прибыли
выручка по второму региону 12415086696.68151
прибыль: (инвестиции) - (выручка по второму региону):  2415086696.681511
процедура Bootstrap
средняя выручка по второму региону - инвестиции 515222773.4432907
нижняя граница 95% интервала прибыль  - инвестиции:  68873225.37050247
верхняя граница 95% интервала прибыль  - инвестиции:  931547591.2570

**Выводы** 
1. В регионах 1 и 3 есть риск убытков - нижняя граница доверительного интервала меньше уровня, необходимого для окупаемости. 
2. По этому критерию 95% доверительного интервала остается для разработки только регион 2. 
3. Дополнительная информация про регион 2: Во втором датасете, по сравнению с другими, суммарные и средние показатели ниже. Но существенно лучше показатель модели RMSE 0.89 (тыс. баррелей) и риск убытков статистически существенно меньше.  